<a href="https://colab.research.google.com/github/kuberiitb/retail_price_ai_agent/blob/main/notebooks/RetailPriceAgent_Text2Sql_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agent created using code from [LangGraph Tutorial](https://python.langchain.com/docs/tutorials/sql_qa/)

In [ ]:
!pip install langchain langchain_community langchain-openai langgraph --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# Setting up SQL data

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///my_database.db")
print(db.get_usable_table_names())

['competitior_price', 'forecast_data', 'historical_data', 'inventory_data']


In [ ]:
print(db.get_table_info(["historical_data"]))


CREATE TABLE historical_data (
	id INTEGER, 
	product TEXT, 
	category TEXT, 
	date TIMESTAMP, 
	price REAL, 
	cost REAL, 
	promotion REAL, 
	seasonality REAL, 
	sales REAL, 
	sales_value REAL, 
	profit REAL
)

/*
3 rows from historical_data table:
id	product	category	date	price	cost	promotion	seasonality	sales	sales_value	profit
1	t-shirt	men	2023-06-01 00:00:00	1157.0	703.0	20.0	1.1	57.0	65949.0	25878.0
1	t-shirt	men	2023-07-01 00:00:00	1163.0	703.0	20.0	1.1	57.0	66291.0	26220.0
1	t-shirt	men	2023-08-01 00:00:00	1446.0	703.0	0.0	1.1	38.0	54948.0	28234.0
*/


In [ ]:
db.run("SELECT * FROM historical_data LIMIT 10;")

"[(1, 't-shirt', 'men', '2023-06-01 00:00:00', 1157.0, 703.0, 20.0, 1.1, 57.0, 65949.0, 25878.0), (1, 't-shirt', 'men', '2023-07-01 00:00:00', 1163.0, 703.0, 20.0, 1.1, 57.0, 66291.0, 26220.0), (1, 't-shirt', 'men', '2023-08-01 00:00:00', 1446.0, 703.0, 0.0, 1.1, 38.0, 54948.0, 28234.0), (1, 't-shirt', 'men', '2023-09-01 00:00:00', 1450.0, 703.0, 0.0, 0.9, 31.0, 44950.0, 23157.0), (1, 't-shirt', 'men', '2023-10-01 00:00:00', 1443.0, 703.0, 0.0, 0.9, 31.0, 44733.0, 22940.0), (1, 't-shirt', 'men', '2023-11-01 00:00:00', 1442.0, 703.0, 0.0, 1.3, 45.0, 64890.0, 33255.0), (1, 't-shirt', 'men', '2023-12-01 00:00:00', 1153.0, 703.0, 20.0, 1.3, 68.0, 78404.0, 30600.0), (1, 't-shirt', 'men', '2024-01-01 00:00:00', 1445.0, 703.0, 0.0, 0.9, 31.0, 44795.0, 23002.0), (1, 't-shirt', 'men', '2024-02-01 00:00:00', 1440.0, 703.0, 0.0, 0.9, 31.0, 44640.0, 22847.0), (1, 't-shirt', 'men', '2024-03-01 00:00:00', 1451.0, 703.0, 0.0, 0.9, 31.0, 44981.0, 23188.0)]"

In [ ]:
db.run("SELECT * FROM competitior_price LIMIT 10;")

"[(1, 't-shirt', 'men', 817.0, 'BOGO', 0.5), (8, 't-shirt', 'women', 968.0, '0.2', 0.8)]"

In [ ]:
db.run("SELECT * FROM forecast_data LIMIT 10;")

"[(1, 't-shirt', 'men', '2025-09-01 00:00:00', 42.0), (1, 't-shirt', 'men', '2025-10-01 00:00:00', 42.0), (1, 't-shirt', 'men', '2025-11-01 00:00:00', 42.0), (1, 't-shirt', 'men', '2025-12-01 00:00:00', 42.0), (1, 't-shirt', 'men', '2026-01-01 00:00:00', 42.0), (1, 't-shirt', 'men', '2026-02-01 00:00:00', 42.0), (8, 't-shirt', 'women', '2025-09-01 00:00:00', 37.0), (8, 't-shirt', 'women', '2025-10-01 00:00:00', 37.0), (8, 't-shirt', 'women', '2025-11-01 00:00:00', 37.0), (8, 't-shirt', 'women', '2025-12-01 00:00:00', 37.0)]"

In [ ]:
db.run("SELECT * FROM inventory_data LIMIT 10;")

"[(1, 't-shirt', 'men', 168.0), (8, 't-shirt', 'women', 74.0)]"

In [ ]:
from sqlalchemy import create_engine, inspect

engine = create_engine("sqlite:///my_database.db")
inspector = inspect(engine)

columns = inspector.get_columns("inventory_data")
for col in columns:
    print(col["name"], col["type"])

In [ ]:
# Langgraph State
from typing_extensions import TypedDict

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

# Setup LLM

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

True

In [ ]:
# from langchain.chat_models import ChatOpenAI

if not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
llm.invoke("Hi").content

'Hello! How can I assist you today?'

# Using inbuilt Langgraph ReAct agent

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for t in tools:
  print(t.name, ":", t.description)

sql_db_query : Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
sql_db_schema : Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
sql_db_list_tables : Input is an empty string, output is a comma-separated list of tables in the database.
sql_db_query_checker : Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.

Business context:
Product usually means product-category combination.
""".format(
    dialect="SQLite",
    top_k=5,
)

sql_agent_executor = create_react_agent(llm, tools, prompt=system_message)

## Example with pretty printing

In [ ]:
question = "Which is highest selling Which is most profitable month?"

for step in sql_agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which is highest selling Which is most profitable month?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_74vZ33Duk1Nj4IsyPS47Fu2A)
 Call ID: call_74vZ33Duk1Nj4IsyPS47Fu2A
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_price, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_9jUmA4ba4mS8ursUZwwMIfgK)
 Call ID: call_9jUmA4ba4mS8ursUZwwMIfgK
  Args:
    table_names: historical_data
  sql_db_schema (call_3Dygzp7g1usrP6yR1JZDPbsb)
 Call ID: call_3Dygzp7g1usrP6yR1JZDPbsb
  Args:
    table_names: forecast_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE forecast_data (
	id INTEGER, 

## Returning final output only

## skipping all the intermediate steps

In [ ]:
def get_sql_reponse(question):
  for step in sql_agent_executor.stream(
      {"messages": [{"role": "user", "content": question}]},
      stream_mode="values",
  ):
      if step['messages'][-1].response_metadata.get('finish_reason')=='stop':
        try:
          return step['messages'][-1].content
        except:
          return "I don't know"

get_sql_reponse("Which customer spent the most?")

"The customers who spent the most are as follows:\n\n1. Helena Holý - $49.62\n2. Richard Cunningham - $47.62\n3. Luis Rojas - $46.62\n4. Ladislav Kovács - $45.62\n5. Hugh O'Reilly - $45.62"

In [ ]:
get_sql_reponse("What is full name of Helena?")

'The full name of Helena is Helena Holý.'